In [ ]:
from modules.PostgresDBHandler import PostgresDBHandler
import os
from joblib import Parallel, delayed
import librosa

In [ ]:
def process_file(file_path, db_params):
    try:
        db = PostgresDBHandler(**db_params)
        db.connect()

        folder_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
        instrument_type = folder_name.split("_")[0]
        microphone_type = os.path.basename(os.path.dirname(file_path))
        audio, sr = librosa.load(file_path, sr=None)

        # Check if the instrument exists in the database
        instrument_id = db.get_instrument_id(instrument_type)

        # Insert the audio file data into the AudioFiles table
        db.insert_audio_file(file_path, sr, len(audio), instrument_id)

        db.close()

        return {
            "instrument_type": instrument_type,
            "microphone_type": microphone_type,
            "original_file": file_path,
            "sample_rate": sr,
        }
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [ ]:
base_dir = "./good-sounds"
dbParams = {
    "dbname": "mydatabase",
    "user": "postgres",
    "password": "password",
    "host": "localhost",
    "port": "5432",
}
# max_samples = 50
n_jobs = -1

In [ ]:
classes = os.listdir(os.path.join(base_dir, "sound_files"))
instrument_types = {className.split("_")[0] for className in classes}

In [ ]:
instrument_types

In [ ]:
db = PostgresDBHandler(**dbParams)
db.connect()
for instrument in instrument_types:
    if db.get_instrument_id(instrument) is None:
        db.insert_instrument(instrument, 0)
db.close()

Load files

In [ ]:
db = PostgresDBHandler(**dbParams) 
db.connect()
audio_files = []
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".wav") and db.check_audio_file_exists(os.path.join(root, file)) is False:
            file_path = os.path.join(root, file)
            audio_files.append(file_path)
db.close()

In [ ]:
print(f"Processing {len(audio_files)} audio files")

In [ ]:
Parallel(n_jobs=n_jobs)(
    delayed(process_file)(file_path, dbParams)
    for file_path in audio_files
)